In [1]:
import pymysql
# 開啟資料庫連線
db = pymysql.connect("chatbot_db","root","123456","" )
# 使用 cursor() 方法建立一個遊標物件 cursor
cursor = db.cursor()
# 使用 execute()  方法執行 SQL 查詢 
cursor.execute("SELECT VERSION()")
# 使用 fetchone() 方法獲取單條資料.
data = cursor.fetchone()
print ("Database version : %s " % data)
# 關閉資料庫連線
db.close()


Database version : 10.1.41-MariaDB-1~bionic 


In [2]:
# test_pic()

In [3]:
'''

整體功能描述

'''

'\n\n整體功能描述\n\n'

In [4]:
'''

Application 主架構

'''

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("./line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [5]:
'''

消息判斷器

讀取指定的json檔案後，把json解析成不同格式的SendMessage

讀取檔案，
把內容轉換成json
將json轉換成消息
放回array中，並把array傳出。

'''

# 引用會用到的套件
from linebot.models import (
    ImagemapSendMessage,TextSendMessage,ImageSendMessage,LocationSendMessage,FlexSendMessage
)

from linebot.models.template import (
    ButtonsTemplate,CarouselTemplate,ConfirmTemplate,ImageCarouselTemplate
    
)

from linebot.models.template import *

def detect_json_array_to_new_message_array(fileName):
    
    #開啟檔案，轉成json
    with open(fileName) as f:
        jsonArray = json.load(f)
    
    # 解析json
    returnArray = []
    for jsonObject in jsonArray:

        # 讀取其用來判斷的元件
        message_type = jsonObject.get('type')
        
        # 轉換
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))    

    # 回傳
    return returnArray

In [6]:
'''

handler處理關注消息

用戶關注時，讀取 素材 -> 關注 -> reply.json

將其轉換成可寄發的消息，傳回給Line

'''

# 引用套件
from linebot.models import (
    FollowEvent
)

# 關注事件處理
@handler.add(FollowEvent)
def process_follow_event(event):
    
    # 讀取並轉換
    result_message_array =[]
    replyJsonPath = "material/關注/reply.json"
    result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
    
    # 呼叫個人用戶名，與用戶第一次接觸打招呼
    user_profile = line_bot_api.get_profile(event.source.user_id)
    hello = {
      "type": "text",
      "text": "哈囉!" + str(user_profile.display_name) + "\n歡迎您使用「履視步爽」"
    }
    result_message_array.insert(0, TextSendMessage.new_from_json_dict(hello))

    # 消息發送
    line_bot_api.reply_message(
        event.reply_token,
        result_message_array
    )

In [7]:
'''

handler處理文字消息

收到用戶回應的文字消息，
按文字消息內容，往素材資料夾中，找尋以該內容命名的資料夾，讀取裡面的reply.json

轉譯json後，將消息回傳給用戶

'''

# 引用套件
from linebot.models import (
    MessageEvent, TextMessage
)

# 文字消息處理
@handler.add(MessageEvent,message=TextMessage)
def process_text_message(event):
    print(event.message.text)
    if '1' in event.message.text:
        pwd = '/root/linechatbot/material/'
        imgWaitToGetPost = [i for i in os.listdir(pwd) if '.jpg' in i]
        pwd = '/root/darknet/result_img'
        imgWaitToGetTest = [i for i in os.listdir(pwd) if '.jpg' in i]
        print(imgWaitToGetPost)
        if len(imgWaitToGetPost) == 0:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='測試中，請稍後輸入「1」確認')
            )
            CopyFileWaitToPost(event.message.id)
        elif len(imgWaitToGetPost) != 0:
            result_message_array = []
            for img in imgWaitToGetPost:
                url = 'https://sneaker.serveo.net/material/'+img
                print(url)
                String = """ImageSendMessage(
                             original_content_url="{}",
                             preview_image_url="{}"
                         )""".format(url, url)
                result_message_array.append(eval(String))
            line_bot_api.reply_message(
                event.reply_token,
                result_message_array
            )
        else:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='請上傳照片')
            )
            
    
    else:
        # 讀取本地檔案，並轉譯成消息
        result_message_array =[]
        replyJsonPath = "material/"+event.message.text+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)

        # 發送
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )


In [8]:
'''

handler處理Postback Event

載入功能選單與啟動特殊功能

解析postback的data，並按照data欄位判斷處理

現有三個欄位
menu, folder, tag

若folder欄位有值，則
    讀取其reply.json，轉譯成消息，並發送

若menu欄位有值，則
    讀取其rich_menu_id，並取得用戶id，將用戶與選單綁定
    讀取其reply.json，轉譯成消息，並發送

'''
from linebot.models import (
    PostbackEvent
)

from urllib.parse import parse_qs 

@handler.add(PostbackEvent)
def process_postback_event(event):
    


    query_string_dict = parse_qs(event.postback.data)
    
    print(query_string_dict)
    if 'folder' in query_string_dict:
    
        result_message_array =[]

        replyJsonPath = 'material/'+query_string_dict.get('folder')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
  
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    elif 'menu' in query_string_dict:
 
        linkRichMenuId = open("material/"+query_string_dict.get('menu')[0]+'/rich_menu_id', 'r').read()
        line_bot_api.link_rich_menu_to_user(event.source.user_id,linkRichMenuId)
        
        replyJsonPath = 'material/'+query_string_dict.get('menu')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
  
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )

In [9]:
'''

若收到圖片消息時，

先回覆用戶文字消息，並從Line上將照片拿回。
擷取主顏色

'''
#!pip install clarifai
# 引用套件
from linebot.models import ImageMessage
from clarifai.rest import Image as ClImage
from clarifai.rest import ClarifaiApp
import os
# from darknet_test import test_pic
from time import sleep


@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
        
    message_content = line_bot_api.get_message_content(event.message.id)
    
    with open('./images/'+event.message.id+'.jpg', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)

    #同時存在一個，上限五則訊息
    line_bot_api.reply_message(
        event.reply_token,
        [TextSendMessage(text='Image has Upload'+ ' ' + event.message.id+"\n"),
         TextSendMessage(text='等待30秒後，請輸入「1」確認')
#          ImageSendMessage(
#             original_content_url=img_url,
#             preview_image_url=img_url
#          )
        ]
    )        
 

    #路徑為相對路徑
    Image_shoes = './images/'+event.message.id+'.jpg'
    
    pwd = '/root/darknet/result_img/'
    for img in [i for i in os.listdir(pwd) if '.jpg' in i]:
        filepath = pwd + img
        if os.path.exists(filepath):
            os.remove(filepath)
    pwd = '/root/linechatbot/material/'
    for img in [i for i in os.listdir(pwd) if '.jpg' in i]:
        filepath = pwd + img
        if os.path.exists(filepath):
            os.remove(filepath)
  
    # 送入darknet檢測照片
    CopyFileWaitToTest(Image_shoes)
    test_pic()
    os.chdir("/root/linechatbot")
#     img_list = CopyFileWaitToPost(event.message.id)
#     img_url ='https://sneaker.serveo.net/material/'+img_list[0]
#     print(img_url)
#     os.chdir("/root/linechatbot")
    
#     imgWaitToGetColor = os.listdir('/root/darknet/result_img')
#     print(imgWaitToGetColor)
    
'''
    # ======================= 官方固定格式 ===============================
    app_color = ClarifaiApp(api_key='0f82436fcad44c1cb8953ee52d563ef1')
    # model = app.public_models.general_model
    model = app_color.models.get('color')
    # picnames = pic_list('D:\clarifai\pictures')
    image = ClImage(filename='/root/darknet/result_img/'+imgWaitToGetColor[-1])
    color_list = model.predict([image])["outputs"][0]["data"]["colors"]
    # ===================================================================
#     print(color_list)
    color_datas = {}
    for color in color_list:
        color_id = color["raw_hex"]
        color_pa = color["value"]
        color_name = color["w3c"]["name"]
#         print("color_data= ",  color_name, color_id + "\t", str(color_pa * 100)[:5] + "%")
        color_datas[color_pa]='main color = '+str(color_name) + " "+ str(color_id) + " " + str(color_pa * 100)[:5] + "%"
#     print(color_datas)
#     print(color_datas.keys())
    max_data = max(color_datas.keys())
    color_data = color_datas[max_data]
    print(color_data)
    
'''


'\n    # ======================= 官方固定格式 ===============================\n    app_color = ClarifaiApp(api_key=\'0f82436fcad44c1cb8953ee52d563ef1\')\n    # model = app.public_models.general_model\n    model = app_color.models.get(\'color\')\n    # picnames = pic_list(\'D:\\clarifai\\pictures\')\n    image = ClImage(filename=\'/root/darknet/result_img/\'+imgWaitToGetColor[-1])\n    color_list = model.predict([image])["outputs"][0]["data"]["colors"]\n    # ===================================================================\n#     print(color_list)\n    color_datas = {}\n    for color in color_list:\n        color_id = color["raw_hex"]\n        color_pa = color["value"]\n        color_name = color["w3c"]["name"]\n#         print("color_data= ",  color_name, color_id + "\t", str(color_pa * 100)[:5] + "%")\n        color_datas[color_pa]=\'main color = \'+str(color_name) + " "+ str(color_id) + " " + str(color_pa * 100)[:5] + "%"\n#     print(color_datas)\n#     print(color_datas.keys())\n    m

In [10]:
from clarifai.rest import Image as ClImage
from clarifai.rest import ClarifaiApp

def GetColor(filePath):
    # ======================= 官方固定格式 ===============================
    app_color = ClarifaiApp(api_key='0f82436fcad44c1cb8953ee52d563ef1')
    # model = app.public_models.general_model
    model = app_color.models.get('color')
    # picnames = pic_list('D:\clarifai\pictures')
    image = ClImage(filename=filePath)
    color_list = model.predict([image])["outputs"][0]["data"]["colors"]
    # ===================================================================
#     print(color_list)
    color_datas = {}
    for color in color_list:
        color_id = color["raw_hex"]
        color_pa = color["value"]
        color_name = color["w3c"]["name"]
#         print("color_data= ",  color_name, color_id + "\t", str(color_pa * 100)[:5] + "%")
        color_datas[color_pa]='main color = '+str(color_name) + " "+ str(color_id) + " " + str(color_pa * 100)[:5] + "%"
#     print(color_datas)
#     print(color_datas.keys())
    max_data = max(color_datas.keys())
    color_data = color_datas[max_data]
    print(color_data)
    return color_data

In [11]:
import os
import shutil

def CopyFileWaitToTest(Image_shoes):
    Image_shoes = Image_shoes.split("/")[-1]
    Image_shoes = "/root/linechatbot/images/" + Image_shoes
    des_pwd = '/root/darknet/data/'
    fn = 'test.jpg'
    shutil.copyfile(Image_shoes, des_pwd+fn)

# CopyFileWaitToTest('./image/10744146872237.jpg')

In [12]:
import os

def test_pic():   
    os.chdir("/root/darknet")
    cmd ="""
        ./darknet detector test \
        shoe_person/cfg/obj.data \
        shoe_person/cfg/yolov3.cfg \
        shoe_person/cfg/weights/yolov3_last.weights \
        data/test.jpg \
        -dont_show &
        """
    os.popen(cmd)
#     cmd_res = os.popen(cmd).read()
#     print(cmd_res)

In [13]:
import os
import shutil

def CopyFileWaitToPost(messageID):
    origin_pwd = '/root/darknet/result_img/'
    des_pwd = '/root/linechatbot/material/'
    img_list = os.listdir(origin_pwd)
    returnImgList = []
    for i in img_list:
        if 'shoe' in i:
            print(i)
            NewfileName = messageID+'-'+i
            shutil.copyfile(origin_pwd+i, des_pwd+NewfileName)
            returnImgList.append(NewfileName)
    return returnImgList

In [ ]:
'''

Application 運行（開發版）

'''
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.25.0.3 - - [16/Oct/2019 05:37:01] "POST / HTTP/1.1" 200 -
172.25.0.3 - - [16/Oct/2019 05:37:02] "POST / HTTP/1.1" 200 -
172.25.0.3 - - [16/Oct/2019 05:37:07] "POST / HTTP/1.1" 200 -


{'tag': ['blockchain']}


172.25.0.3 - - [16/Oct/2019 05:37:09] "POST / HTTP/1.1" 200 -


想知道區塊鏈的戰略價值


172.25.0.3 - - [16/Oct/2019 05:37:14] "POST / HTTP/1.1" 200 -
172.25.0.3 - - [16/Oct/2019 05:37:18] "POST / HTTP/1.1" 200 -


...

172.25.0.3 - - [16/Oct/2019 05:47:57] "GET /material/10749816103631-img_1_1_0_shoe.jpg HTTP/1.1" 200 -
172.25.0.3 - - [16/Oct/2019 05:47:57] "GET /material/10749816103631-img_1_3_0_shoe.jpg HTTP/1.1" 200 -
172.25.0.3 - - [16/Oct/2019 05:51:22] "POST / HTTP/1.1" 200 -
172.25.0.3 - - [16/Oct/2019 05:51:54] "POST / HTTP/1.1" 200 -


1
[]
img_1_2_0_shoe.jpg
img_1_1_0_shoe.jpg


172.25.0.3 - - [16/Oct/2019 05:52:00] "POST / HTTP/1.1" 200 -


1
['10749833451113-img_1_2_0_shoe.jpg', '10749833451113-img_1_1_0_shoe.jpg']
https://sneaker.serveo.net/material/10749833451113-img_1_2_0_shoe.jpg
https://sneaker.serveo.net/material/10749833451113-img_1_1_0_shoe.jpg


172.25.0.3 - - [16/Oct/2019 05:52:02] "GET /material/10749833451113-img_1_2_0_shoe.jpg HTTP/1.1" 200 -
172.25.0.3 - - [16/Oct/2019 05:52:02] "GET /material/10749833451113-img_1_1_0_shoe.jpg HTTP/1.1" 200 -


In [ ]:
'''

Application 運行（heroku版）

'''

# import os
# if __name__ == "__main__":
#     app.run(host='0.0.0.0',port=os.environ['PORT'])